# Домашнее задание к вебинару №4. Тематическое моделирование

In [7]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-09 10:28:10--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-09 10:28:11--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucca0e34eaf396f65a2fe3fe4a83.dl.dropboxusercontent.com/cd/0/inline/Bm3I6Zdyk3mFMBc5JJeb09rg8a226DS3rQgyfTxPBYJbYp99ZPUAJXT2MWsC2aUEMyOhRCkIUdSjCY6ViLCUD_0CQkvWFUsyFd-e0m65nJQ8NccjdA30JZtpIDkGRb0vBWrZr01ObI_Z3HSKKlkl6wX0zo7huSFqb-2K4Oiu9-yuVw/file# [following]
--2022-06-09 10:28:11--  https://ucca0e34eaf396f65a2fe3fe4a83.dl.dropboxusercontent.com/cd/0/inline/Bm3I6Zdyk3mFMBc5JJeb09rg8a226DS3rQgyfTxPBYJbYp99ZPUAJXT2MWsC2aUEMyOh

Постановка задачи:
1. Объединить в одну выборку
2. Провести исследование и выявить тематики о которых говорят в твитах
3. Сделать визуализацию кластеров тематик
4. Проинтерпритировать получившиеся тематики

In [1]:
# pip install pymorphy2

In [2]:
# pip install pyLDAvis

In [3]:
# pip install stop_words

In [135]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from pymystem3 import Mystem
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
import string
from pymorphy2 import MorphAnalyzer
from gensim.models import *
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [136]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['tweet'])

negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['tweet'])

# объединим датасеты:
df = positive.append(negative)

In [137]:
df.head(3)

,tweet
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...


**Предобработка текста**

In [138]:
# Лемматизация

morpher = MorphAnalyzer()
sw = get_stop_words("ru") + stopwords.words('russian')
punkt = list(string.punctuation)
noise = set(sw + punkt)

# Создадим функцию для токенизации, лемматизации и удаления слишком коротких токенво (короче 3 символов) и небукв
def lemmatize_text(text):
    return [t for t in [morpher.parse(token)[0].normal_form 
                                for token in word_tokenize(text)] if t not in noise and t.isalpha() and len(t) >= 4]

In [143]:
# очистка токенов от ненужных для определения темы и нецензурных слов

def clear_tokens(tokens):
    token_str = ' '.join(tokens)
    token_str = re.sub(r'[A-za-z]', ' ',  token_str) 
    token_str = re.sub(r'\w+ще\b', ' ',  token_str) 
    token_str = re.sub(r'\bбл\w+', ' ',  token_str) 
    token_str = re.sub(r'\bидио\w+', ' ',  token_str) 
    token_str = re.sub(r'\bпид\w+', ' ',  token_str)
    token_str = re.sub(r'\bху\w+', ' ', token_str)
    token_str = re.sub(r'\w+xу.\b', ' ',  token_str)
    token_str = re.sub(r'\w+xу\w+', ' ',  token_str)
    token_str = re.sub(r'\w+хо\b', ' ',  token_str)
    token_str = re.sub(r'\bсук\w+', ' ',  token_str)
    token_str = re.sub(r'\bпоп\w+', ' ',  token_str)
    token_str = re.sub(r'\bпиз\w+', ' ',  token_str)
    token_str = re.sub(r'\bгов\w+', ' ',  token_str)
    token_str = re.sub(r'\bфиг\w+', ' ',  token_str)
    token_str = re.sub(r'\bпи.де\w+', ' ',  token_str)
    token_str = re.sub(r'никто',  ' ', token_str) 
    token_str = re.sub(r'завтра',  ' ', token_str) 
    token_str = re.sub(r'свой', ' ', token_str) 
    token_str = re.sub(r'\w+ть\b', ' ',  token_str) 
    token_str = re.sub(r'\w+ться\b', ' ',  token_str)
    token_str = re.sub(r'который', ' ', token_str)
    token_str = re.sub(r'нету', ' ', token_str)
    token_str = re.sub(r'вчера', ' ', token_str)
    token_str = re.sub(r'\w+ти\b', ' ', token_str)
    token_str = re.sub(r'самый', ' ', token_str)
    clear_tokens = token_str.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize_text(text)))

In [144]:
df['tweet'] = df['tweet'].apply(preprocess)
df.head()

,tweet
0,"[школотый, поверь, общество, предмет]"
1,"[похожий, мальчик, равно, хороший]"
2,[]
3,"[угол, голод, порция]"
4,"[страшилка, ощущение, автор]"


**Применение модели LDA для разделения твитов на тематики**

In [145]:
# создание словаря (индекc - токен)
dictionary = corpora.Dictionary(df['tweet'])

In [146]:
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 

dictionary.save('tweet.dict')

len(dictionary.keys()) #длина словаря из токенов

7107

In [147]:
# Векторизуем твиты 

corpus = [dictionary.doc2bow(tweet) for tweet in df['tweet']]
corpora.MmCorpus.serialize('tweets.model', corpus) 

In [148]:
print(df['tweet'].iloc[1])
print(corpus[1])

['похожий', 'мальчик', 'равно', 'хороший']
[(4, 1), (5, 1), (6, 1), (7, 1)]


In [149]:
# Обучим модель LDA

lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3)

In [154]:
lda.show_topics(num_topics=3, num_words=25, formatted=True)

[(0,
  '0.023*"большой" + 0.016*"новый" + 0.015*"жаль" + 0.014*"последний" + 0.014*"настроение" + 0.012*"ночь" + 0.011*"неделя" + 0.009*"каникулы" + 0.009*"улица" + 0.009*"телефон" + 0.009*"урок" + 0.009*"жалко" + 0.008*"снег" + 0.008*"маленький" + 0.007*"деньга" + 0.007*"город" + 0.007*"учёба" + 0.006*"месяц" + 0.006*"холодно" + 0.006*"мама" + 0.006*"выходной" + 0.006*"фильм" + 0.006*"домой" + 0.006*"должный" + 0.006*"папа"'),
 (1,
  '0.026*"хороший" + 0.018*"утро" + 0.014*"любимый" + 0.014*"голова" + 0.010*"ужасно" + 0.008*"сильно" + 0.008*"твиттер" + 0.008*"нужный" + 0.008*"фотка" + 0.008*"вечер" + 0.008*"смочь" + 0.008*"чувство" + 0.008*"нога" + 0.007*"боль" + 0.007*"мама" + 0.007*"грустно" + 0.007*"момент" + 0.006*"никак" + 0.006*"песня" + 0.006*"добрый" + 0.006*"прям" + 0.006*"класс" + 0.006*"лето" + 0.005*"старый" + 0.005*"целый"'),
 (2,
  '0.025*"школа" + 0.014*"друг" + 0.012*"работа" + 0.010*"скоро" + 0.010*"дело" + 0.010*"плохой" + 0.009*"экзамен" + 0.009*"равно" + 0.008*"при

**Оценка модели**

In [151]:
# 1. Перплексия - чем ниже , тем лучше 
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  0.00031437170088904074


In [152]:
# 2. Средняя когерентность - чем выше, тем лучше
coherence_model_lda = CoherenceModel(model=lda, texts=df['tweet'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Средняя когерентность: ', coherence_lda)

Средняя когерентность:  0.27644322465375104


**Визуализация результатов модели**



In [153]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

Эмпирически мной было подобрано оптимальное количество тематик = 3.  
При классификации на бОльшее количестве тем, на визуализации 2 и более тематик пересекаются друг с другом (расположены слишком близко)
Можно выделить темы: 1 - досуг/планирование времени, 2 - чувства/впечателения/личное мнение, 3 - образование и работа
